# Theory Framework Classification Using Section Classifier Framework

This notebook demonstrates the use of the new section classifier framework for analyzing theoretical frameworks in physics education research articles.



## Initial Setup

In [2]:
import pandas as pd

# Add parent directory to path to import local modules
# sys.path.insert(0, r"C:/Users/sfgar/programing/INTED-article-splitting/__computational_essays/python_code")

from LLM_classifier import (
    load_categories_from_json,
    save_categories_to_json,
    create_timestamped_path,
    CostEstimationWrapper
)
from theory_classifiers import FrameworkClassifier

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from python_code import helpers

from IPython.display import Markdown


In [3]:

api_key = helpers.safe_load_env_variable("OPENAI_API_KEY", "../.env.secret")
# Initialize the classifier with loaded frameworks
classifier = FrameworkClassifier(
    api_key = api_key,
    temperature=0.6,
    max_tokens=15000,
    general_model="gpt-4.1-mini",
    reasoning_model="o3-mini"
)

## Data loading

In [ ]:
# Load the data
df = pd.read_pickle("../section_type_classification/data/classified_sections_light_gpt-4.1-mini_20250716_122412.pkl")

# Load existing theory categories
frameworks = load_categories_from_json("./theory_data/reviewed_categories_20250721_163329.json")

Successfully loaded 135 categories from ./reviewed_categories_20250721_163329.json


## Filter Theoretical Framework Sections

In [4]:

# Helper function to filter DataFrame by category
def filter_df_by_category(
    df,
    value: str,
    column: str = "classification_highest_prob_gpt-4.1-mini"
):
    """
    Returns a DataFrame filtered to only rows where the specified column matches the given value.
    """
    mask = df[column] == value
    return df[mask].copy()

# Filter for theoretical framework sections
theory_df = filter_df_by_category(df, "Theoretical Framework")
print(f"Found {len(theory_df)} theoretical framework sections")

Found 589 theoretical framework sections


## Batch Classify Sections

#### Estimate costs

In [6]:
cost_estimator = CostEstimationWrapper(classifier)

data = classifier._df_to_sections(theory_df)

est = cost_estimator.estimate_full_workflow(data, frameworks)

display(Markdown(cost_estimator.generate_cost_report(est)))

# Cost Estimation Report
Generated: 2025-08-06 00:21:54

## Category Review

- **Cost**: $0.0029
- **Model**: o3-mini
- **Input Tokens**: 6,589
- **Output Tokens**: 3,170

## Batch Classification

- **Total Cost**: $1.3243
- **Items Processed**: 589
- **Average Cost per Item**: $0.0022
- **Total Input Tokens**: 5,108,903
- **Total Output Tokens**: 47,120

## Summary

- **Total Estimated Cost**: $1.3272
- **Total Input Tokens**: 5,115,492
- **Total Output Tokens**: 50,290

*Note: These are estimates based on current pricing and may vary.*

#### Do the actual classification

Run the classification. the `category_creation_temperature` parampeter decides how aggressive the model is in creating new categories.

In [5]:

# Prepare the DataFrame for batch classification
# theory_batch_df = theory_df[['article_id', 'section_title', 'section_content']].copy()

# Process a subset of sections for testing
# test_size = 2
# print(f"\nProcessing {test_size} sections as a test...")

# Run batch classification
result_df, updated_categories = classifier.batch_classify_sections_df(theory_df, frameworks, category_creation_temperature="locked")


Processing element 1/589
Input tokens used: 10897
Output tokens used: 312
Processing element 2/589
Input tokens used: 8171
Output tokens used: 298
Processing element 3/589
Input tokens used: 7453
Output tokens used: 307
Processing element 4/589
Input tokens used: 8160
Output tokens used: 149
Processing element 5/589
Input tokens used: 7584
Output tokens used: 141
Processing element 6/589
Input tokens used: 8749
Output tokens used: 272
Classifications contains values that does not exist as category: ['Conceptual Change Theory']
Probability distribution contains keys that are not available category: ['Conceptual Change Theory']
Retrying 2/5...
Input tokens used: 8785
Output tokens used: 329
Classifications contains values that does not exist as category: ['Conceptual Change Theory']
Probability distribution contains keys that are not available category: ['Conceptual Change Theory']
Retrying 3/5...
Input tokens used: 8821
Output tokens used: 283
Classifications contains values that does n

Let us preview the results to make sure they make sense.

In [8]:
result_df

,article_id,section_title,classifications_gpt-4.1-mini,probabilities_gpt-4.1-mini,highest_prob_gpt-4.1-mini,text_excerpts_gpt-4.1-mini
0,10.1103/PhysRevSTPER.2.010103,THEORETICAL FRAME: A MODEL OF COGNITION,"[Resource Framing, Knowledge Integration Frame...","{'Resource Framing': 0.6, 'Knowledge Integrati...",Resource Framing,{'Resource Framing': [{'excerpt': 'We are inte...
1,10.1103/PhysRevSTPER.2.010103,STUDENT MODEL SPACE: A MATHEMATICAL REPRESENTA...,"[Mathematical Modeling Framework, Postpositivism]","{'Mathematical Modeling Framework': 0.85, 'Pos...",Mathematical Modeling Framework,{'Mathematical Modeling Framework': [{'excerpt...
2,10.1103/PhysRevSTPER.2.010105,BACKGROUND AND VALIDITY OF BEMA,"[Measurement Theory Framework, Formative Asses...","{'Measurement Theory Framework': 0.75, 'Format...",Measurement Theory Framework,{'Measurement Theory Framework': [{'excerpt': ...
3,10.1103/PhysRevSTPER.2.020101,THEORETICAL FRAMES,"[Modeling, Structure Mapping, and Conceptual B...","{'Modeling, Structure Mapping, and Conceptual ...","Modeling, Structure Mapping, and Conceptual Bl...","{'Modeling, Structure Mapping, and Conceptual ..."
4,10.1103/PhysRevSTPER.2.020103,DEFINING SCIENTIFIC ABILITIES,[Formative Assessment Framework (Assessment fo...,{'Formative Assessment Framework (Assessment f...,Formative Assessment Framework (Assessment for...,{'Formative Assessment Framework (Assessment f...
...,...,...,...,...,...,...
584,10.1103/PhysRevPhysEducRes.20.020145,BACKGROUND,"[Philosophy & History of Science Framework, Ep...",{'Philosophy & History of Science Framework': ...,Philosophy & History of Science Framework,{'Philosophy & History of Science Framework': ...
585,10.1103/PhysRevPhysEducRes.20.020146,THE COMMOGNITIVE THEORY,[Commognitive Theory],{'Commognitive Theory': 1.0},Commognitive Theory,{'Commognitive Theory': [{'excerpt': 'Rooted i...
586,10.1103/PhysRevPhysEducRes.20.020146,ANALYSIS OF SCIENTIFIC CONTENT: CONCEPTUALIZAT...,"[Model of Educational Reconstruction (MER), Co...",{'Model of Educational Reconstruction (MER)': ...,Model of Educational Reconstruction (MER),{'Model of Educational Reconstruction (MER)': ...
587,10.1103/PhysRevPhysEducRes.20.020147,THEORETICAL BACKGROUND,[Johnson-Laird Mental Representation Framework...,{'Johnson-Laird Mental Representation Framewor...,Johnson-Laird Mental Representation Framework,{'Johnson-Laird Mental Representation Framewor...


The classificator also extracts some text exerpts from the classification data. Currently this is not used for anything. But it could be used e.g. for generating improved and more detailed descriptions of theoretical frameworks by passing a bunch of exerpts for the same framework to a LLM.

Here we print out a sample of them for a simple overview of what it extracts.

In [7]:
import json

# Print out the text excerpts from 3 random rows using sample
sampled = result_df.sample(3, random_state=42)
for idx, row in sampled.iterrows():
    excerpts = row.get("text_excerpts_gpt-4.1-mini", [])
    # If excerpts is a string, try to parse as JSON
    if isinstance(excerpts, str):
        try:
            excerpts = json.loads(excerpts)
        except Exception:
            pass  # If it can't be parsed, leave as is
    if not excerpts or excerpts == "No excerpts found":
        print(f"\nRow {idx}: No excerpts found")
        continue
    print(f"\nRow {idx} excerpts by category:")
    for excerpt in excerpts:
        # If excerpt is a string, try to parse as JSON
        if isinstance(excerpt, str):
            try:
                excerpt = json.loads(excerpt)
            except Exception:
                excerpt = {}
        category = excerpt.get("category", "Unknown Category")
        text = excerpt.get("excerpt", "No excerpt text")
        print(f"  Category: {category}\n    Excerpt: {text}")


Row 521 excerpts by category:
  Category: Unknown Category
    Excerpt: No excerpt text

Row 284 excerpts by category:
  Category: Unknown Category
    Excerpt: No excerpt text
  Category: Unknown Category
    Excerpt: No excerpt text
  Category: Unknown Category
    Excerpt: No excerpt text

Row 513 excerpts by category:
  Category: Unknown Category
    Excerpt: No excerpt text
  Category: Unknown Category
    Excerpt: No excerpt text


#### Classification and new category processing and saving

In [9]:
len(updated_categories)

135

In [ ]:

# Save updated categories if any new ones were discovered
if len(updated_categories) > len(frameworks):
    print(f"\nDiscovered {len(updated_categories) - len(frameworks)} new categories")
    save_categories_to_json(updated_categories, "./theory_data/reviewed_categories.json")

In [11]:
# Merge result_df columns into theory_df using 'article_id' and 'section_title' for alignment, prefixing with 'theory_'
merge_cols = [col for col in result_df.columns if col not in ['article_id', 'section_title']]
theory_df_merged = theory_df.merge(
    result_df.rename(columns={col: f"theory_{col}" for col in merge_cols}),
    on=['article_id', 'section_title'],
    how='left'
)

In [12]:
import numpy as np

# Count the rows in theory_df_merged with a NaN value in 'theory_classification_gpt_4.1-mini'
nan_count = theory_df_merged['theory_probabilities_gpt-4.1-mini'].isna().sum()
print(f"Number of rows with NaN in 'theory_classification_gpt_4.1-mini': {nan_count}")


Number of rows with NaN in 'theory_classification_gpt_4.1-mini': 0


In [13]:
theory_df_merged

,article_abstract,article_articleType,article_authors,article_affiliations,article_date,article_type,article_metadata_last_modified_at,article_last_modified_at,article_id,article_identifiers,...,section_title_embedding,section_content_embedding,probability_dist_gpt-4.1-mini,classification_gpt-4.1-mini,classification_highest_prob_gpt-4.1-mini,failed_validation_gpt-4.1-mini,theory_classifications_gpt-4.1-mini,theory_probabilities_gpt-4.1-mini,theory_highest_prob_gpt-4.1-mini,theory_text_excerpts_gpt-4.1-mini
0,{'value': '<p>Decades of education research ha...,article,"[{'type': 'Person', 'name': 'Lei Bao', 'firstn...","[{'name': 'Department of Physics, The Ohio Sta...",2006-02-02,article,2006-03-02T15:44:54+0000,2014-08-22 11:32:06+00:00,10.1103/PhysRevSTPER.2.010103,{'doi': '10.1103/PhysRevSTPER.2.010103'},...,"[-0.05480332672595978, 0.024843383580446243, 0...","[-0.06690618395805359, 0.023494919762015343, 0...","{'Theoretical Framework': 0.9, 'Literature Rev...",[Theoretical Framework],Theoretical Framework,False,"[Resource Framing, Knowledge Integration Frame...","{'Resource Framing': 0.6, 'Knowledge Integrati...",Resource Framing,{'Resource Framing': [{'excerpt': 'We are inte...
1,{'value': '<p>Decades of education research ha...,article,"[{'type': 'Person', 'name': 'Lei Bao', 'firstn...","[{'name': 'Department of Physics, The Ohio Sta...",2006-02-02,article,2006-03-02T15:44:54+0000,2014-08-22 11:32:06+00:00,10.1103/PhysRevSTPER.2.010103,{'doi': '10.1103/PhysRevSTPER.2.010103'},...,"[-0.064679816365242, 0.023139355704188347, 0.0...","[-0.07020589709281921, 0.03362146019935608, 0....","{'Theoretical Framework': 0.6, 'Methods': 0.35...","[Theoretical Framework, Methods]",Theoretical Framework,False,"[Mathematical Modeling Framework, Postpositivism]","{'Mathematical Modeling Framework': 0.85, 'Pos...",Mathematical Modeling Framework,{'Mathematical Modeling Framework': [{'excerpt...
2,{'value': '<p>The Brief Electricity and Magnet...,article,"[{'type': 'Person', 'name': 'Lin Ding', 'first...","[{'name': 'Department of Physics, North Caroli...",2006-03-15,article,2006-03-15T15:52:03+0000,2014-08-22 04:01:44+00:00,10.1103/PhysRevSTPER.2.010105,{'doi': '10.1103/PhysRevSTPER.2.010105'},...,"[0.0021336465142667294, 0.039801325649023056, ...","[-0.0529276579618454, 0.04701779782772064, 0.0...","{'Introduction / Motivation': 0.05, 'Theoretic...","[Theoretical Framework, Methods]",Theoretical Framework,False,"[Measurement Theory Framework, Formative Asses...","{'Measurement Theory Framework': 0.75, 'Format...",Measurement Theory Framework,{'Measurement Theory Framework': [{'excerpt': ...
3,{'value': '<p>Previous studies have demonstrat...,article,"[{'type': 'Person', 'name': 'Noah S. Podolefsk...","[{'name': 'Department of Physics, University o...",2006-07-18,article,2006-07-18T17:56:03+0000,2014-08-22 04:01:46+00:00,10.1103/PhysRevSTPER.2.020101,{'doi': '10.1103/PhysRevSTPER.2.020101'},...,"[-0.05005525052547455, -0.011595192365348339, ...","[-0.07176671177148819, -0.0012164521031081676,...","{'Theoretical Framework': 0.9, 'Literature Rev...",[Theoretical Framework],Theoretical Framework,False,"[Modeling, Structure Mapping, and Conceptual B...","{'Modeling, Structure Mapping, and Conceptual ...","Modeling, Structure Mapping, and Conceptual Bl...","{'Modeling, Structure Mapping, and Conceptual ..."
4,{'value': '<p>The paper introduces a set of fo...,article,"[{'type': 'Person', 'name': 'Eugenia Etkina', ...",[{'name': 'Department of Physics and Astronomy...,2006-08-01,article,2006-08-01T19:34:02+0000,2014-08-22 04:19:22+00:00,10.1103/PhysRevSTPER.2.020103,{'doi': '10.1103/PhysRevSTPER.2.020103'},...,"[-0.04178931564092636, 0.032831691205501556, -...","[-0.07951419055461884, 0.03167480602860451, -0...","{'Introduction / Motivation': 0.1, 'Theoretica...",[Theoretical Framework],Theoretical Framework,False,[Formative Assessment Framework (Assessment fo...,{'Formative Assessment Framework (Assessment f...,Formative Assessment Framework (Ass

In [ ]:
helpers.save_processed_embeddings(theory_df_merged, "./theory_data/theory_classification_df")

Saved processed_embeddings to theory_classification_df_20250806_192015.pkl


'theory_classification_df_20250806_192015.pkl'

## Review categories

### Load classified data (to avoid rerunning)

In [ ]:
theory_df_merged = pd.read_pickle("./theory_data/theory_classification_df_20250806_192015.pkl")
updated_categories = load_categories_from_json("./theory_data/reviewed_categories_20250721_163329.json")
print(theory_df_merged.keys())
print(len(updated_categories))

Successfully loaded 135 categories from ./reviewed_categories_20250721_163329.json
Index(['article_abstract', 'article_articleType', 'article_authors',
       'article_affiliations', 'article_date', 'article_type',
       'article_metadata_last_modified_at', 'article_last_modified_at',
       'article_id', 'article_identifiers', 'article_issue',
       'article_pageStart', 'article_hasArticleId', 'article_numPages',
       'article_publisher', 'article_rights', 'article_journal',
       'article_title', 'article_volume', 'article_notes',
       'article_tocSection', 'article_fundings',
       'article_classificationSchemes', 'article_doi', 'article_full_text_xml',
       'article_full_text', 'article_year', 'section_relative_position',
       'section_label', 'section_title', 'section_content', 'section_id',
       'section_title_embedding', 'section_content_embedding',
       'probability_dist_gpt-4.1-mini', 'classification_gpt-4.1-mini',
       'classification_highest_prob_gpt-4.1-mi

### Run Cleaning

In [21]:
reviewed_categories = classifier.review_and_clean_categories(updated_categories)

Category review using model: o3-mini
Reviewing 144 categories
Input tokens used: 7284
Output tokens used: 6774
Review summary:
  - Original categories: 144
  - Cleaned categories: 118
  - Removed categories: 16
  - Merge suggestions: 7
  - Removed: Curriculum Documents Framework, Participatory Design Framework, Practice-Based Teacher Education, Postpositivism, Epistemic Network Analysis Framework, Innovation-decision Process Framework, Human Capital Theory, How Learning Works (HLW) Principles Framework, Adaptation and Reinvention (ARI) Model, Philosophy & History of Science Framework, Organizational Theory in Educational Contexts, Design-Based Research (DBR) Methodology, Research Ethics and Questionable Research Practices in Science Education, Pragmatism in Research, Interpretivist Case Study, Natural Language Processing Framework
  - Suggested merges: Sociocultural Participation and Situated Learning Framework, Critical Race Theoretical Perspectives, Constructivism Framework, Cultural

In [24]:
from IPython.display import Markdown
issues = classifier._validate_review_result(reviewed_categories, updated_categories)
report = classifier.generate_validation_report(reviewed_categories, updated_categories, issues)
with open("review_report.md", "w") as f:
    f.write(report)
# display(Markdown(report))


In [ ]:
save_categories_to_json(reviewed_categories.cleaned_categories, "./theory_data/reviewed_categories.json")

Categories saved to reviewed_categories_20250721_163329.json


## Create Meta-Categories

In [ ]:
from theory_classifiers import FrameworkClassifier


classifier = FrameworkClassifier(api_key=api_key, temperature=0.5,
                                max_tokens=1000, general_model="gpt-4.1", reasoning_model="o3-mini")
                                # OBS! We are using gpt-4.1 here because of the low token length

In [ ]:
frameworks = load_categories_from_json("./theory_data/reviewed_categories_20250721_163329.json")

Successfully loaded 135 categories from reviewed_categories_20250721_163329.json


#### Discover/load initial categories

The classifier can pass the raw data, in this case the framework titles and descriptions, to a LLM to discover a initial set of categories.

In [7]:
preped_discovery_data = [f"{cat.title}: {cat.description}" for cat in frameworks]
# discovered_meta_categories = classifier.discover_categories(preped_discovery_data)

Let us instead load the initial categories suggested by Tor Ole and print them out for a overview.

In [ ]:
meta_categories = load_categories_from_json("./theory_data/meta_categories_TOOB.json")

Successfully loaded 5 categories from meta_categories_TOOB.json


In [9]:

for x in meta_categories:
    print(x.title)
    print(x.description)
    print("-"*100)

Cognitive
Theories focusing on individual mental processes, knowledge structures, and reasoning. Examples include Symbolic forms, epistemic games, Dual process theory, and misconceptions research.
----------------------------------------------------------------------------------------------------
Sociocultural
Theories emphasizing learning as social participation and enculturation into a community's practices. Examples include Cultural-Historical Activity Theory (CHAT), Participationist learning, and Communities of Practice.
----------------------------------------------------------------------------------------------------
Social Justice
Theories that examine and critique systems of power, privilege, and oppression within educational contexts. Examples include Critical Race Theory and Culturally Relevant Pedagogy.
----------------------------------------------------------------------------------------------------
Organizational
Theories that analyze the structures, systems, and proces

#### Review and clean the initial categories

The classifier can also review and clean up the categories. Since the amount of tokens used for this process is negligible in the case of meta-categorization, we can run it on the initial categories just for the sake of it.

In [10]:
review_response = classifier.review_and_clean_categories(meta_categories)
reviewed_meta_cat = review_response.cleaned_categories

Category review using model: o3-mini
Reviewing 5 categories


Input tokens used: 800
Output tokens used: 829
Review summary:
  - Original categories: 5
  - Cleaned categories: 5
  - Removed categories: 0
  - Merge suggestions: 0


We can also save these if we'd like.

In [11]:
# save_categories_to_json(reviewed_meta_cat, "reviewed_meta_cat.json")

### Run meta-categorization

We can now run the classification on the frameworks using this set of meta-categories.

#### Cost estimate

Let us first estimate the cost of running our classifier with the selected models.

In [12]:
cost_classifier = CostEstimationWrapper(classifier)

estimates = cost_classifier.estimate_full_workflow(frameworks, reviewed_meta_cat, preped_discovery_data)
display(Markdown(cost_classifier.generate_cost_report(estimates)))


# Cost Estimation Report
Generated: 2025-08-06 23:35:44

## Category Discovery

- **Cost**: $0.0219
- **Model**: gpt-4.1
- **Input Tokens**: 7,619
- **Output Tokens**: 830

## Category Review

- **Cost**: $0.0004
- **Model**: o3-mini
- **Input Tokens**: 525
- **Output Tokens**: 470

## Batch Classification

- **Total Cost**: $0.2255
- **Items Processed**: 135
- **Average Cost per Item**: $0.0017
- **Total Input Tokens**: 69,546
- **Total Output Tokens**: 10,800

## Summary

- **Total Estimated Cost**: $0.2477
- **Total Input Tokens**: 77,690
- **Total Output Tokens**: 12,100

*Note: These are estimates based on current pricing and may vary.*

#### Classifying

In [13]:
classification_response = classifier.batch_classify(frameworks, reviewed_meta_cat, "balanced")

Processing element 1/135
Input tokens used: 603
Output tokens used: 53
Processing element 2/135
Input tokens used: 601
Output tokens used: 54
Processing element 3/135
Input tokens used: 598
Output tokens used: 52
Processing element 4/135
Input tokens used: 600
Output tokens used: 54
Processing element 5/135
Input tokens used: 603
Output tokens used: 48
Processing element 6/135
Input tokens used: 608
Output tokens used: 57
Processing element 7/135
Input tokens used: 610
Output tokens used: 55
Processing element 8/135
Input tokens used: 601
Output tokens used: 34
Processing element 9/135
Input tokens used: 612
Output tokens used: 48
Processing element 10/135
Input tokens used: 632
Output tokens used: 52
Processing element 11/135
Input tokens used: 611
Output tokens used: 55
Processing element 12/135
Input tokens used: 610
Output tokens used: 48
Processing element 13/135
Input tokens used: 627
Output tokens used: 62
Processing element 14/135
Input tokens used: 610
Output tokens used: 48
P

In [14]:

classification = classification_response[0]
extended_categories = classification_response[1]

len(extended_categories)
categories_diff = len(extended_categories) - len(meta_categories)
print(f"{categories_diff} new meta-categories generated during classification!")

1 new meta-categories generated during classification!


### Analyzing

With these classifications of the frameworks, we can now see if we can find any patterns using these meta-categories.

Let us first add a new column to our dataframe. For this, we need a mapping from frameworks to meta-categories.

In [18]:
category_mapping = {}

for cat in classification:
    cat_title = cat[0].title
    cat_prob = cat[1].probabilities  # This is a list of ProbabilityScore objects
    # Find the ProbabilityScore object with the highest probability
    max_prob_obj = max(cat_prob, key=lambda p: p.probability)
    category_mapping[cat_title] = max_prob_obj.category

# Now, map the old column to the new one
# theory_df_merged['classification_meta_framework_category'] = theory_df_merged['theory_highest_prob_gpt-4.1-mini'].map(category_mapping.get)
theory_df_merged['classification_meta_framework_category'] = theory_df_merged['theory_classifications_gpt-4.1-mini'].map(lambda r: [category_mapping.get(el) for el in r])

In [ ]:
theory_df_merged['theory_classifications_gpt-4.1-mini']

0      [Resource Framing, Knowledge Integration Frame...
1                      [Mathematical Modeling Framework]
2      [Measurement Theory Framework, Formative Asses...
3      [Modeling, Structure Mapping, and Conceptual B...
4                       [Formative Assessment Framework]
                             ...                        
584          [Philosophy & History of Science Framework]
585                                [Commognitive Theory]
586    [Model of Educational Reconstruction (MER), Co...
587    [Mental Models Framework, Mathematical Modelin...
588    [Sociocultural Learning Theory, Metacognition ...
Name: theory_classifications_gpt-4.1-mini, Length: 589, dtype: object

In [ ]:
reversed_map = {}

for key,val in category_mapping.items():
    reversed_map.setdefault(val, []).append(key)

with open(create_timestamped_path("./theory_data/category_map.json"), "w") as fp:
    json.dump(reversed_map, fp)


# pd.DataFrame({"Meta Category":reversed_map.keys(), "Frameworks": reversed_map.values()})

Using this, we can now plot the development over time.

In [22]:
import plotly.express as px
from LLM_classifier.utils import create_timestamped_path
import pandas as pd # It's good practice to import pandas

# Ensure the relevant columns exist
category_col = "classification_meta_framework_category"
year_col = 'article_year'

# 1. Start with the original DataFrame and drop rows with NaN values
plot_df = theory_df_merged.dropna(subset=[category_col, year_col]).copy()

# 2. Filter out rows where the category_col list is empty.
#    The condition is changed to > 0 to KEEP rows with non-empty lists.
is_not_empty_mask = plot_df[category_col].str.len() > 0
plot_df = plot_df[is_not_empty_mask].copy()

print(f"Number of sections: {len(plot_df)}")

# 3. THE KEY STEP: Explode the DataFrame.
#    This creates a new row for each category in the list, repeating the year.
plot_df = plot_df.explode(category_col)

print(f"Number of exploded entries: {len(plot_df)}")

# 4. Convert year to integer
plot_df[year_col] = plot_df[year_col].astype(int)

# 5. Get category counts. This groupby now works correctly on the exploded data.
category_counts = (
    plot_df.groupby([year_col, category_col])
    .size()
    .reset_index(name='count')
)

# 6. Create interactive plot (This part remains the same)
fig = px.line(
    category_counts,
    x=year_col,
    y='count',
    color=category_col,
    markers=True,
    labels={
        year_col: 'Year',
        'count': 'Number of Sections',
        category_col: 'Theory Category'
    },
    title='Theory Category Counts by Year (Exploded Data)'
)

fig.update_layout(
    legend_title_text='Theory Category',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.01
    ),
    width=1000,
    height=500
)

# output_path = create_timestamped_path("plotly_theory_year_exploded.html")
# fig.write_html(output_path)
# print(f"Plot saved to: {output_path}")
fig.show()


Number of sections: 582
Number of exploded entries: 1310
